In [2]:
from google.colab import drive

drive.mount("/content/drive")

Mounted at /content/drive


In [3]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [4]:
def predict(X_train, P, T):
  # Interpolate once to arrive at the correct P
  P_lower, P_upper = find_bounds(X_train, 0, P)
  P_data = interpolate(P_lower, P_upper, P, 0, 1)
  
  # Second interpolation to arrive at both correct P and T
  T_lower, T_upper = find_bounds(P_data, 1, T)
  properties = interpolate(T_lower, T_upper, T, 1, 0)

  # Return
  return properties[0]

def find_bounds(data, index, value):
  # Three cases, value is smaller than all of the values in the index column, 
  # value is contained by a pair of values in the index column, 
  # or value is greater than or equal to all the values in the index column
  unique_values = np.unique(data[:, index])
  unique_values = np.sort(unique_values)

  # if the value is less than or equal to the smallest value in the column, 
  # return all the entries with the smallest first two values
  if value <= unique_values[0]:
    lower = data[data[:, index] == unique_values[0]]
    upper = data[data[:, index] == unique_values[1]]
  elif value >= unique_values[-2]:
    lower = data[data[:, index] == unique_values[-2]]
    upper = data[data[:, index] == unique_values[-1]]
  else:
    for i in range(len(unique_values)):
      if value < unique_values[i]:
        lower = data[data[:, index] == unique_values[i-1]]
        upper = data[data[:, index] == unique_values[i]]
        break

  return lower, upper

def interpolate(lower, upper, value, value_index, keep_index):
  # Determine the set of keep values lower and upper share in common
  _, lower_index, upper_index = np.intersect1d(lower[:, keep_index], upper[:, keep_index], return_indices=True)

  # Iterate through all the shared values
  result = []
  for i in range(len(lower_index)):
    l_i = lower_index[i]
    u_i = upper_index[i]
    
    interp_entry = (value - lower[l_i, value_index]) / (upper[u_i, value_index] - lower[l_i, value_index]) * (upper[u_i, :] - lower[l_i, :]) + lower[l_i, :]
    result.append(interp_entry)
  
  # Return
  return np.array(result)

# Import data
data = pd.read_csv("drive/My Drive/H2O_Super.csv")
X_train, X_test = train_test_split(data.values, train_size=0.8, random_state=1)

# Calculate predictions
X_predict = np.zeros((np.size(X_test, 0), np.size(X_test, 1)))
for i in range(np.size(X_test, 0)):
  X_predict[i, :] = predict(X_train, X_test[i, 0], X_test[i, 1])

# Compute normalized SSE
X_mean = np.mean(X_train, 0)
X_std = np.std(X_train, 0)

X_test = (X_test - X_mean) / X_std
X_predict = (X_predict - X_mean) / X_std

error = X_test - X_predict
SSE = np.sum(error ** 2)
print(SSE)

1.2890607016797917
